In [1]:
# importing Required libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

# Structure-1 (Same as given Notebook)

In [2]:
class Net(nn.Module): # image siE in mnist is 1x28x28
                                           


    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # 1 convolutional block
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
# Checking that cuda is available or not
torch.cuda.is_available()

True

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.99245023727417 batch_id=468: 100%|██████████| 469/469 [00:39<00:00, 11.95it/s]



Test set: Average loss: 1.9687, Accuracy: 2790/10000 (28%)



# Structure-2 (Except structure all are same)

In [27]:
class Net(nn.Module): # image siE in mnist is 1x28x28
                                           
## Without max pooling, use kernels with different shapes and randomly increase as well as decreasing the filter size.

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)    #  32x28x28.
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)   #  64x28x28.
        #self.pool1 = nn.MaxPool2d(2, 2)               #  Cancelling the maxpool part.
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  #  128x28x28.
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #  256x28x28.
        #self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)            #  512x26x26
        self.conv6 = nn.Conv2d(512, 1024, 5)           #  1024x22x22
        self.conv7 = nn.Conv2d(1024, 2048, 7)          #  2048x16x16
        self.conv8 = nn.Conv2d(2048, 512, 3)           #  512x14x14
        self.conv9 = nn.Conv2d(512, 1024, 7)           #  1024x8x8
        self.conv10 = nn.Conv2d(1024, 2048, 5)         #  2048x4x4
        self.conv11 = nn.Conv2d(2048, 1024, 3)         #  1024x2x2
        self.conv12 = nn.Conv2d(1024, 10, 2)           #  10x1x1

    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x))))   # 1 convolutional block, self.pool1(
        x = F.relu(self.conv4(F.relu(self.conv3(x))))   #self.pool2(
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv8(F.relu(self.conv7(x))))
        x = F.relu(self.conv10(F.relu(self.conv9(x))))
        x = F.relu(self.conv12(F.relu(self.conv11(x))))
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [28]:
# Checking that cuda is available or not
torch.cuda.is_available()

True

In [29]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Conv2d-3          [-1, 128, 28, 28]          73,856
            Conv2d-4          [-1, 256, 28, 28]         295,168
            Conv2d-5          [-1, 512, 26, 26]       1,180,160
            Conv2d-6         [-1, 1024, 22, 22]      13,108,224
            Conv2d-7         [-1, 2048, 16, 16]     102,762,496
            Conv2d-8          [-1, 512, 14, 14]       9,437,696
            Conv2d-9           [-1, 1024, 8, 8]      25,691,136
           Conv2d-10           [-1, 2048, 4, 4]      52,430,848
           Conv2d-11           [-1, 1024, 2, 2]      18,875,392
           Conv2d-12             [-1, 10, 1, 1]          40,970
Total params: 223,914,762
Trainable params: 223,914,762
Non-trainable params: 0
-----------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [30]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [31]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [32]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   ## It is taking more and moere time to train. I think because of the
                                                                   ## filters dimension as i chnaged them randomly and lot of number of channels such as 2048, 1024.
                                                                   ## I decrease them and also increase them in a random manner.
                                                                   ## The next reason may be i did not use the maxpooling and use the kernels randomly 
                                                                   ## Such as 3x3, 5x5, 7x7. As kernels are high in dimension with high number of filters,
                                                                   ## this will be the reason for taking so much time to train.

''' it is taking --> "2h 1m 7s" to train and give the prediction. The accuracy is also
decrease.'''


for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=2.3036651611328125 batch_id=468: 100%|██████████| 469/469 [1:54:34<00:00, 14.66s/it]



Test set: Average loss: 2.2995, Accuracy: 1135/10000 (11%)



# Structure-3

In [54]:
class Net(nn.Module): # image siE in mnist is 1x28x28
                                           
# More maxpooling and more padding.

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)               ## 32x28x28
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)              ## 64x28x28
        self.pool1 = nn.MaxPool2d(2, 2)                           ## 64x14x14
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)             ## 128x14x14
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)            ## 256x14x14
        self.pool2 = nn.MaxPool2d(4, 2)                           ## 256x6x6
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)            ## 512x6x6
        self.conv6 = nn.Conv2d(512, 1024, 3, padding = 1)         ## 1024x6x6  
        
        #self.pool3 = nn.MaxPool2d(2, 2)                           ## 1024x5x5
        
        self.conv7 = nn.Conv2d(1024, 512, 3, padding = 1)         ## 512x6x6
        self.pool3 =  nn.MaxPool2d(2, 1)                          ## 512x5x5
        self.conv8 = nn.Conv2d(512, 1024, 3)                      ## 1024x3x3
        self.conv9 = nn.Conv2d(1024, 10, 3)                       ## 10x1x1



    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # 1 convolutional block
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.pool3(F.relu(self.conv7(x)))
        x = F.relu(self.conv9(F.relu(self.conv8(x))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [55]:
# Checking that cuda is available or not
torch.cuda.is_available()

True

In [56]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 6, 6]               0
            Conv2d-7            [-1, 512, 6, 6]       1,180,160
            Conv2d-8           [-1, 1024, 6, 6]       4,719,616
            Conv2d-9            [-1, 512, 6, 6]       4,719,104
        MaxPool2d-10            [-1, 512, 5, 5]               0
           Conv2d-11           [-1, 1024, 3, 3]       4,719,616
           Conv2d-12             [-1, 10, 1, 1]          92,170
Total params: 15,818,506
Trainable params: 15,818,506
Non-trainable params: 0
-------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [57]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [58]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [59]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   


for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.8178346753120422 batch_id=468: 100%|██████████| 469/469 [01:17<00:00,  6.05it/s]



Test set: Average loss: 0.7775, Accuracy: 7778/10000 (78%)



# Structure - 4

In [36]:
class Net(nn.Module): # image siE in mnist is 1x28x28
                                           
# More maxpooling, bigger kernel size as well as padding size

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 7, padding=3)         ## 32x28x28
        self.conv2 = nn.Conv2d(32, 64, 7, padding=3)        ## 64x28x28
        self.pool1 = nn.MaxPool2d(2, 2)                     ## 64x14x14
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)       ## 128x14x14
        self.conv4 = nn.Conv2d(128, 256, 7, padding=3)      ## 256x14x14
        self.pool2 = nn.MaxPool2d(4, 2)                     ## 256x6x6
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)      ## 512x6x6
        self.conv6 = nn.Conv2d(512, 1024, 3, padding=1)     ## 1024x6x6
        self.pool3 = nn.MaxPool2d(2, 2)                     ## 1024x3x3
        self.conv7 = nn.Conv2d(1024, 10, 3)                 ## 10x1x1

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) # 1 convolutional block
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.pool3(F.relu(self.conv6(F.relu(self.conv5(x)))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [37]:
# Checking that cuda is available or not
torch.cuda.is_available()

True

In [38]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           1,600
            Conv2d-2           [-1, 64, 28, 28]         100,416
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]       1,605,888
         MaxPool2d-6            [-1, 256, 6, 6]               0
            Conv2d-7            [-1, 512, 6, 6]       1,180,160
            Conv2d-8           [-1, 1024, 6, 6]       4,719,616
         MaxPool2d-9           [-1, 1024, 3, 3]               0
           Conv2d-10             [-1, 10, 1, 1]          92,170
Total params: 7,773,706
Trainable params: 7,773,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.81
Params size (MB): 29.65
Estima

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [39]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [40]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [41]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)   

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.3390775918960571 batch_id=468: 100%|██████████| 469/469 [01:25<00:00,  5.48it/s]



Test set: Average loss: 1.2694, Accuracy: 5661/10000 (57%)

